In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import chess.pgn as pgn
import chess
import numpy as np
from math import ceil

In [2]:
data=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_games.csv')
data.Scores=data.Scores.str.replace(' ','').str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Scores=data.Scores.apply(lambda x: list(map(int,x.split())))
data.Scores=data.Scores.apply(lambda x: list(map(int,x)))
data.Moves=data.Moves.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.Moves=data.Moves.apply(lambda x: x.split())
data.UCI=data.UCI.str.replace('"','').str.replace("'","").str.replace('[','').str.replace(']','').str.replace(',',' ')
data.UCI=data.UCI.apply(lambda x: x.split())
data.head()

,Result,Moves,UCI,Scores,NumMoves,MeanScore,ModeScore,FinalScore
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...",19,23.947368,3,54
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...",53,-692.198113,18,-11544
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...",39,49.246753,31,196
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[e2e4, c7c5, g1f3, d7d6, b2b4, g8f6, b4c5, f6e...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...",25,306.040816,18,875
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[g1f3, d7d5, d2d4, g8f6, c2c4, e7e6, b1c3, f8e...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...",29,30.827586,18,-11


In [3]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,57,4765
1,1915,1999,1957,84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
features=data.copy()

'''
# Может попробовать так? (долго думает, пока непонятно надо это или нет)
wins = pd.DataFrame(columns = ["WhiteWins", "BlackWins"])
features=features.append(wins)
features.update(features[["WhiteWins", "BlackWins"]].fillna(0, inplace=False), overwrite=False)

for i,x in enumerate(features['Result']):
    if (x == "1/2-1/2"):
        features['WhiteWins'][i] = 1
        features['BlackWins'][i] = 1
    elif (x == "1-0"):
        features['WhiteWins'][i] = 1
    else:
        features['BlackWins'][i] = 1
        '''

features.Result=features.Result.apply(lambda x: 0.5 if x=='1/2-1/2' else x)
features.Result=features.Result.apply(lambda x: 1 if x=='1-0' else x)
features.Result=features.Result.apply(lambda x: 0 if x=='0-1' else x)

features=features.drop(columns=['Moves','Scores','UCI'])
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore
0,0.5,19,23.947368,3,54
1,0.0,53,-692.198113,18,-11544
2,1.0,39,49.246753,31,196
3,1.0,25,306.040816,18,875
4,0.5,29,30.827586,18,-11


#### Общие оценки

In [5]:
features['MinScore']=data.Scores.apply(lambda x: min(x))
features['MaxScore']=data.Scores.apply(lambda x: max(x))
features['MedianScore']=data.Scores.apply(lambda x: np.median(x))
features['StdDevScore']=data.Scores.apply(lambda x: np.std(x))
features['VarianceScore']=data.Scores.apply(lambda x: round(np.var(x),2))
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore,VarianceScore
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587,698.63
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735,5226398.31
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093,2654.84
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595,98134.08
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827,617.32


#### Общая разница в оценках

In [6]:
def delta(x,func):
    diffs=[]
    for i in range(len(x)-1):
        diffs.append((x[i+1]-x[i]))
    if func=='mean':
        return np.mean(diffs)
    elif func=='min':
        return min(diffs)
    elif func=='max':
        return max(diffs)
    elif func=='median':
        return np.median(diffs)
    elif func=='std':
        return np.std(diffs)
    elif func=='var':
        return np.var(diffs)

features['MeanDeltaScore']=data.Scores.apply(delta,args=('mean',))
features['MinDeltaScore']=data.Scores.apply(delta,args=('min',))
features['MaxDeltaScore']=data.Scores.apply(delta,args=('max',))
features['MedianDeltaScore']=data.Scores.apply(delta,args=('median',))
features['StdDevDeltaScore']=data.Scores.apply(delta,args=('std',))
features['VarianceDeltaScore']=data.Scores.apply(delta,args=('var',))
features.head()

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore,VarianceScore,MeanDeltaScore,MinDeltaScore,MaxDeltaScore,MedianDeltaScore,StdDevDeltaScore,VarianceDeltaScore
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587,698.63,0.972973,-27,44,-4.0,16.203914,2.625668e+02
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735,5226398.31,-110.190476,-9578,5009,-4.0,1090.554329,1.189309e+06
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093,2654.84,2.552632,-69,85,-1.0,25.355055,6.428788e+02
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595,98134.08,17.687500,-87,309,5.0,67.765575,4.592173e+03
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827,617.32,-0.508772,-57,29,1.0,15.034607,2.260394e+02


#### Признаки только по отдельным цветам

In [7]:
data['WhiteMoves']=data.Moves.apply(lambda x: x[::2])
data['WhiteMovesUCI']=data.UCI.apply(lambda x: x[::2])
data['WhiteScores']=data.Scores.apply(lambda x: x[::2])
data['BlackMoves']=data.Moves.apply(lambda x: x[1::2])
data['BlackMovesUCI']=data.UCI.apply(lambda x: x[::2])
data['BlackScores']=data.Scores.apply(lambda x: x[1::2])
data=data.drop(columns=['NumMoves','MeanScore','ModeScore','FinalScore'])
data.head()

,Result,Moves,UCI,Scores,WhiteMoves,WhiteMovesUCI,WhiteScores,BlackMoves,BlackMovesUCI,BlackScores
0,1/2-1/2,"[Nf3, Nf6, c4, c5, b3, g6, Bb2, Bg7, e3, O-O, ...","[g1f3, g8f6, c2c4, c7c5, b2b3, g7g6, c1b2, f8g...","[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,...","[Nf3, c4, b3, Bb2, e3, Be2, O-O, Nc3, Qc2, Rac...","[g1f3, c2c4, b2b3, c1b2, e2e3, f1e2, e1g1, b1c...","[18, 12, -5, 3, 22, 20, 8, 11, -6, 1, -21, -26...","[Nf6, c5, g6, Bg7, O-O, b6, Bb7, Nc6, Rc8, d5,...","[g1f3, c2c4, b2b3, c1b2, e2e3, f1e2, e1g1, b1c...","[17, 8, 12, -2, 21, 13, 21, 3, 5, -10, -1, 18,..."
1,0-1,"[e4, d5, exd5, Nf6, d4, Nxd5, Nf3, g6, Be2, Bg...","[e2e4, d7d5, e4d5, g8f6, d2d4, f6d5, g1f3, g7g...","[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5...","[e4, exd5, d4, Nf3, Be2, c4, Nc3, O-O, Be3, d5...","[e2e4, e4d5, d2d4, g1f3, f1e2, c2c4, b1c3, e1g...","[26, 68, 65, 48, 61, 63, 53, 69, 30, -2, -11, ...","[d5, Nf6, Nxd5, g6, Bg7, Nb6, O-O, Nc6, Bg4, B...","[e2e4, e4d5, d2d4, g1f3, f1e2, c2c4, b1c3, e1g...","[51, 57, 77, 93, 63, 58, 46, 29, 27, 12, 0, 24..."
2,1-0,"[c4, Nf6, Nc3, d6, d4, e5, Nf3, Nbd7, Bg5, Be7...","[c2c4, g8f6, b1c3, d7d6, d2d4, e7e5, g1f3, b8d...","[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ...","[c4, Nc3, d4, Nf3, Bg5, e3, Qc2, Bd3, h4, dxe5...","[c2c4, b1c3, d2d4, g1f3, c1g5, e2e3, d1c2, f1d...","[2, 5, 35, 37, 10, 8, 30, 13, -12, 13, -22, -1...","[Nf6, d6, e5, Nbd7, Be7, c6, O-O, h6, b5, dxe5...","[c2c4, b1c3, d2d4, g1f3, c1g5, e2e3, d1c2, f1d...","[21, 53, 45, 54, 22, 48, 17, 35, 31, 31, 20, -..."
3,1-0,"[e4, c5, Nf3, d6, b4, Nf6, bxc5, Nxe4, cxd6, Q...","[e2e4, c7c5, g1f3, d7d6, b2b4, g8f6, b4c5, f6e...","[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9...","[e4, Nf3, b4, bxc5, cxd6, d4, Bd3, O-O, Be3, B...","[e2e4, g1f3, b2b4, b4c5, c5d6, d2d4, f1d3, e1g...","[26, 35, 18, 18, 10, 60, 91, 83, 58, 98, 146, ...","[c5, d6, Nf6, Nxe4, Qb6, Bg4, Nxd6, Nc6, Nb4, ...","[e2e4, g1f3, b2b4, b4c5, c5d6, d2d4, f1d3, e1g...","[64, 53, 20, 20, 49, 95, 82, 93, 59, 142, 193,..."
4,1/2-1/2,"[Nf3, d5, d4, Nf6, c4, e6, Nc3, Be7, Bg5, h6, ...","[g1f3, d7d5, d2d4, g8f6, c2c4, e7e6, b1c3, f8e...","[18, 29, 10, 22, 20, 18, 19, 28, 24, 18, 17, 1...","[Nf3, d4, c4, Nc3, Bg5, Bh4, e3, Bxe7, Qc2, Qx...","[g1f3, d2d4, c2c4, b1c3, c1g5, g5h4, e2e3, h4e...","[18, 10, 20, 19, 24, 17, 18, 48, 51, 44, 41, 2...","[d5, Nf6, e6, Be7, h6, O-O, Ne4, Qxe7, Nxc3, d...","[g1f3, d2d4, c2c4, b1c3, c1g5, g5h4, e2e3, h4e...","[29, 22, 18, 28, 18, 16, 38, 37, 53, 48, 16, 1..."


In [8]:
features['MinWhiteScore']=data.WhiteScores.apply(lambda x: min(x))
features['MaxWhiteScore']=data.WhiteScores.apply(lambda x: max(x))
features['MedianWhiteScore']=data.WhiteScores.apply(lambda x: np.median(x))
features['StdDevWhiteScore']=data.WhiteScores.apply(lambda x: np.std(x))
features['MeanWhiteScore']=data.WhiteScores.apply(lambda x: np.mean(x))
features['ModeWhiteScore']=data.WhiteScores.apply(lambda x: max(set(x), key=x.count))
features['FinalWhiteScore']=data.WhiteScores.apply(lambda x: x[-1])
features['VarianceWhiteScore']=data.WhiteScores.apply(lambda x: np.var(x))

features['MinBlackScore']=data.BlackScores.apply(lambda x: min(x))
features['MaxBlackScore']=data.BlackScores.apply(lambda x: max(x))
features['MedianBlackScore']=data.BlackScores.apply(lambda x: np.median(x))
features['StdDevBlackScore']=data.BlackScores.apply(lambda x: np.std(x))
features['MeanBlackScore']=data.BlackScores.apply(lambda x: np.mean(x))
features['ModeBlackScore']=data.BlackScores.apply(lambda x: max(set(x), key=x.count))
features['FinalBlackScore']=data.BlackScores.apply(lambda x: x[-1])
features['VarianceBlackScore']=data.BlackScores.apply(lambda x: np.var(x))

features.shape

(24255, 32)

#### Разница в оценках по цветам

In [9]:
features['MeanWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('mean',))
features['MinWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('min',))
features['MaxWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('max',))
features['MedianWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('median',))
features['StdDevWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('std',))
features['VarianceWhiteDeltaScore']=data.WhiteScores.apply(delta,args=('var',))

features['MeanBlackDeltaScore']=data.BlackScores.apply(delta,args=('mean',))
features['MinBlackDeltaScore']=data.BlackScores.apply(delta,args=('min',))
features['MaxBlackDeltaScore']=data.BlackScores.apply(delta,args=('max',))
features['MedianBlackDeltaScore']=data.BlackScores.apply(delta,args=('median',))
features['StdDevBlackDeltaScore']=data.BlackScores.apply(delta,args=('std',))
features['VarianceWhiteDeltaScore']=data.BlackScores.apply(delta,args=('var',))

features.shape

(24255, 43)

#### Индекс первого хода, когда белые или черные получили преимущество (>100 cp)
+победили ли они после получения преимущества (если не было преимущества - то сверяем с ничьей)

In [10]:
def find_advantage(scores):
    for i in range(len(scores)):
        if abs(scores[i]) >=100:
            return i
    return 0

def is_realised_advantage(ind,result):
    if ind==0 and result==0.5:
        return 1
    elif ind!=0 and ind %2!=0 and result==1:
        return 1
    elif ind!=0 and ind %2==0 and result==0:
        return 1
    else:
        return 0        
    
features['FirstAdvantageInd']=data.Scores.apply(find_advantage)
features['IsAdvantageRealised']=features[['FirstAdvantageInd','Result']].apply(lambda x: is_realised_advantage(*x),axis=1)
features['FirstAdvantageInd']=features['FirstAdvantageInd'].apply(lambda x: ceil(x/2))
features.head(10)

,Result,NumMoves,MeanScore,ModeScore,FinalScore,MinScore,MaxScore,MedianScore,StdDevScore,VarianceScore,...,MedianWhiteDeltaScore,StdDevWhiteDeltaScore,VarianceWhiteDeltaScore,MeanBlackDeltaScore,MinBlackDeltaScore,MaxBlackDeltaScore,MedianBlackDeltaScore,StdDevBlackDeltaScore,FirstAdvantageInd,IsAdvantageRealised
0,0.5,19,23.947368,3,54,-26,73,18.0,26.431587,698.63,...,0.5,22.114531,2.221636e+02,2.055556,-18,30,-1.5,14.905153,0,1
1,0.0,53,-692.198113,18,-11544,-11544,93,-48.0,2286.131735,5226398.31,...,-9.0,767.218965,2.304473e+06,-222.980769,-9685,3830,-5.0,1518.048956,29,1
2,1.0,39,49.246753,31,196,-30,198,37.0,51.525093,2654.84,...,2.0,25.605087,5.713630e+02,4.351351,-50,68,3.0,23.903201,33,1
3,1.0,25,306.040816,18,875,10,917,142.0,313.263595,98134.08,...,16.0,78.221274,7.070737e+03,34.956522,-64,286,16.0,84.087676,10,1
4,0.5,29,30.827586,18,-11,-15,94,29.0,24.845827,617.32,...,1.0,21.645421,3.266020e+02,-1.428571,-53,34,-1.5,18.072134,0,1
5,1.0,38,237.440000,32,898,-4,6395,104.0,744.029556,553579.98,...,4.0,65.416695,8.268580e+05,177.333333,-65,5539,9.0,909.317326,14,1
6,0.5,64,-1081.496063,0,0,-10293,64,-100.0,2132.508389,4547592.03,...,-2.0,848.578524,3.357153e+06,-1.032258,-5372,10293,-2.0,1832.253495,17,0
7,1.0,46,106.923077,37,410,-3,445,83.0,93.119943,8671.32,...,3.0,28.120850,2.769109e+03,6.068182,-149,251,0.0,52.622324,31,1
8,1.0,30,139.033898,22,351,-5,420,100.0,119.455540,14269.63,...,13.0,35.291280,1.443321e+03,13.500000,-32,160,5.0,37.991070,12,0
9,1.0,38,253.746667,39,1240,15,1581,65.0,345.365356,119277.23,...,10.0,74.848618,8.648490e+03,41.305556,-35,378,11.5,92.997258,22,1


In [ ]:
# Не мое, просто попробовать
features = features.append(pd.DataFrame(columns = ["WhiteAvg", "BlackAvg", "WhiteStdDev", "BlackStdDev"]))

for i,x in enumerate(data["Scores"]):
    white_moves = x[::2]
    black_moves = x[1::2]
    # Calculate White Deltas
    if len(black_moves) < len(white_moves):
        black_moves = np.insert(black_moves,0,0)
    else:
        black_moves = np.insert(black_moves,0,0)
        white_moves = np.append(white_moves, black_moves[-1])
    features["WhiteAvg"][i] = np.mean(np.subtract(white_moves, black_moves))
    features["WhiteStdDev"][i] = np.std(np.subtract(white_moves, black_moves))
    
    #Calculate Black Deltas
    white_moves = x[::2]
    if len(x) % 2 == 0:
        black_moves = x[1::2]
    else:
        black_moves = np.append(x[1::2], white_moves[-1])
    features["BlackAvg"][i] = - np.mean(np.subtract(black_moves, white_moves))
    features["BlackStdDev"][i] = np.std(np.subtract(black_moves, white_moves))
    
features.shape

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of

# НЕ ТЕСТИЛОСЬ ДАЛЬШЕ НИЧЕГО

### А ЭТО ПРОСТО ТЕСТ СМОГУ ЛИ Я ТАК СДЕЛАТЬ

In [ ]:
game = pgn.Game()
node = game.add_variation(chess.Move.from_uci(data.UCI[0][0]))
for move in data.UCI[0][1:]:
    node = node.add_variation(chess.Move.from_uci(move))

print(game)

In [ ]:
print(data.Moves[0])

# Подумать и добавить еще признаков (мб по цветам)
### (и назвать нормально)

In [ ]:
def extract_game_features(uci):
    game = pgn.Game()
    node = game.add_variation(chess.Move.from_uci(uci[0]))
    for move in uci[1:]:
        node = node.add_variation(chess.Move.from_uci(move))

    first_check = True
    first_queen_move = True

    features = defaultdict(int)

    while node.variations:
        move = node.variation(0).move

        board = node.board()

        # какая фигура пошла на этом ходу и что съела
        moved_piece = board.piece_type_at(move.from_square)
        captured_piece = board.piece_type_at(move.to_square)
        
        # если это был ферзь (на каком ходу)
        if moved_piece == QUEEN and first_queen_move:
            features['queen_moved_at'] = board.fullmove_number
            first_queen_move = False
            
        # если съели ферзя (на каком ходу)
        if captured_piece == QUEEN:
            features['queen_changed_at'] = board.fullmove_number

        if move.promotion: # превращение пешки (сколько пешек превратилось)
            features['promotion'] += 1
        if board.is_check(): # количество шахов в партии
            features['total_checks'] += 1
            if first_check:
                features['first_check_at'] = board.fullmove_number
                first_check = False
                
        # рокировка (на каком ходу)
        if board.is_kingside_castling(move) and board.turn== WHITE:
            features['white_king_castle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== WHITE:
            features['white_queen_castle'] = board.fullmove_number
        elif if board.is_kingside_castling(move) and board.turn== BLACK:
            features['black_king_castle'] = board.fullmove_number
        elif board.is_queenside_castling(move) and board.turn== BLACK:
            features['black_queen_castle'] = board.fullmove_number
            
        # количество взятий на проходе
        if board.is_en_passant(move): 
            features['en_passant'] += 1
            
        node = node.variation(0)
        
    # анализ итоговой позиции 
    if board.is_checkmate(): #шах
        features['is_checkmate'] += 1
    if board.is_stalemate(): # мат 
        features['is_stalemate'] += 1
    # проверяет, есть ли у одной из сторон достаточно материала для победы (false, если, например, один конь остался)
    if board.is_insufficient_material():
        features['insufficient_material'] += 1
    # Проверка на правило 50-ти ходов или троекратное повторение
    if board.can_claim_draw():
        features['can_claim_draw'] += 1

    # Pieces at the end of the game
    piece_placement = board.fen().split()[0]
    end_pieces = Counter(x for x in piece_placement if x.isalpha())

    # count number of piece at end position
    features.update({'end_' + piece: cnt
                     for piece, cnt in end_pieces.items()})
    return features

# Поизучать DictVectorize и заменить некоторые свои функции на более удобные

In [ ]:
def games_features(games, return_names=False):
    l = []
    for game in games:
        features = extract_game_features(game)
        l.append(features)

    vec = DictVectorizer()
    X = vec.fit_transform(l)
    if return_names:
        return X, vec.get_feature_names()
    else:
        return X


def extract_score_features(scores):
    features = dict()
    if scores.size == 0:
        return features
    scores = np.r_[0, scores]
    abs_scores = np.abs(scores)
    diffs = np.diff(scores)
    white_diffs = diffs[::2]
    black_diffs = diffs[1::2]
    subset_names = ['diffs', 'white_diffs', 'black_diffs']
    subsets = [diffs, white_diffs, black_diffs]
    stats = [np.min, np.max, np.mean, lambda x: np.median(np.abs(x))]
    stat_names = ['min', 'max', 'mean', 'median_abs']
    for subset, subset_name in zip(subsets, subset_names):
        for stat, stat_name in zip(stats, stat_names):
            features[stat_name + '_' + subset_name] = stat(subset)
            # np.hi
    features['advantage120_idx'] = np.argmax(abs_scores > 120) or len(scores)
    features['advantage70_idx'] = np.argmax(abs_scores > 70) or len(scores)
    return features


def score_features(stockfish_scores):
    l = stockfish_scores.MoveScores.apply(extract_score_features).tolist()
    vec = DictVectorizer()
    X = vec.fit_transform(l)
    df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
    return df

#### Совершались ли "грубые" ошибки? (Изменение в оценке позиции > 100) 
+ход, на котором была совершена грубая ошибка

+поиск количества ошибок (Количество раз, когда оценка менялась более, чем на 50)

In [ ]:
for i in range(len(data.Scores[1])):
    print("i={} :{}".format(i,data.Scores[1][i]))

In [ ]:
def delta(x):
    diffs=[]
    for i in range(len(x)-1):
        diffs.append((x[i+1]-x[i]))
        print("i={} - {} :{}".format(i+1,i,diffs[i]))

diffs=delta(data.Scores[1])
print(diffs)

In [ ]:
def find_erros(x):
    count_erros=0
    count_blunders=0
    for i in range(len(x)-1):
        if abs(x[i+1]-x[i]) >=100:
            count_blunders+=1
            is_blunders=1
            ind_first_blunders=i
        if abs(x[i+1]-x[i]) >=50:
            count_erros+=1
            is_erros=1
            ind_first_erros=i
    return diffs

diffs=delta(data.Scores[1])

# Условные разделения на дебют, миттельшпиль и эндшпиль. И подсчет оценок внутри разделений

In [ ]:
data = data.append(pd.DataFrame(columns = ["DebScores", "MitScores", "EndScores"]))

# Разделения довольно условные, но имеют основания, так как игроки могут по-разному показывать себя в разных частях партии.
data['DebScores']=data.Scores.apply(lambda x: x[:16])
data['MitScores']=data.Scores.apply(lambda x: x[16:36])
data['EndScores']=data.Scores.apply(lambda x: x[36:])

'''
1.	white part mean
2.	black part mean
3.	white part stddev
4.	black part sttdev
5.	white part max
6.	balck part max

'''

### Кто предложил ничью?

In [ ]:
for ind,x in enumerate(features['Result']):
    if x==0.5:
        if len(data.Moves) % 2==0:
            features['offer_draw']=0 # код для черных
        else:
            features['offer_draw']=1 # код для белых
    else:
        features['offer_draw']=-1 # никто не предлагал

In [ ]:
features.to_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv', sep=',',index=False)